In [1]:
!pip install transformers datasets accelerate
# accelerate is often needed for Trainer

In [2]:
from datasets import load_dataset
dataset = load_dataset("DialogueCharacter/english_general_instruction_with_reward_score_judged_by_13B_llama2")

README.md:   0%|          | 0.00/576 [00:00<?, ?B/s]

data/train-00000-of-00007-ae75d20b7a29eb(…):   0%|          | 0.00/254M [00:00<?, ?B/s]

data/train-00001-of-00007-36c97018b8253a(…):   0%|          | 0.00/281M [00:00<?, ?B/s]

data/train-00002-of-00007-e9b4501c65cb37(…):   0%|          | 0.00/276M [00:00<?, ?B/s]

data/train-00003-of-00007-3ced0fdd39228c(…):   0%|          | 0.00/264M [00:00<?, ?B/s]

data/train-00004-of-00007-d960b2dd57945e(…):   0%|          | 0.00/243M [00:00<?, ?B/s]

data/train-00005-of-00007-4ce4d5fe294abe(…):   0%|          | 0.00/197M [00:00<?, ?B/s]

data/train-00006-of-00007-f165108ec74aa4(…):   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1006809 [00:00<?, ? examples/s]

In [3]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token # GPT-2 typically uses the EOS token for padding

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [7]:
def format_and_tokenize(examples):
    # Customize this template based on your dataset structure
    formatted_texts = []
    for instruction, response in zip(examples['input'], examples['output']):
        # Example template (adjust as needed):
        text = f"input: {instruction}\noutput: {output}{tokenizer.eos_token}"
        formatted_texts.append(text)

    # Tokenize the batch of formatted texts
    return tokenizer(
        formatted_texts,
        truncation=True,
        max_length=512, # Adjust max_length based on your resources and data
        padding='max_length'
    )

tokenized_datasets = dataset.map(format_and_tokenize, batched=True, remove_columns=['input', 'output', 'reward_score'])

# Set the 'input_ids' as 'labels' for Causal Language Modeling
tokenized_datasets = tokenized_datasets.map(lambda examples: {'labels': examples['input_ids']}, batched=True)
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#

Map:   0%|          | 0/1006809 [00:00<?, ? examples/s]

NameError: name 'output' is not defined

In [4]:
def format_and_tokenize(examples):
    # Customize this template based on your dataset structure
    formatted_texts = []

    # 1. Define loop variables as 'instruction' and 'response'
    for instruction, response in zip(examples['input'], examples['output']):

        # 2. ERROR HERE: Changed 'output' to the defined variable 'response'
        text = f"input: {instruction}\noutput: {response}{tokenizer.eos_token}"

        formatted_texts.append(text)

    # Tokenize the batch of formatted texts
    return tokenizer(
        formatted_texts,
        truncation=True,
        max_length=512, # Adjust max_length based on your resources and data
        padding='max_length'
    )

tokenized_datasets = dataset.map(format_and_tokenize, batched=True, remove_columns=['input', 'output', 'reward_score'])

# Set the 'input_ids' as 'labels' for Causal Language Modeling
tokenized_datasets = tokenized_datasets.map(lambda examples: {'labels': examples['input_ids']}, batched=True)

Map:   0%|          | 0/1006809 [00:00<?, ? examples/s]

Map:   0%|          | 0/1006809 [00:00<?, ? examples/s]

In [5]:
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained("gpt2")
# You might want to use a smaller model like 'gpt2-medium' if you run out of GPU memory.

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [6]:
from transformers import TrainingArguments, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./gpt2_instruction_finetuned",
    overwrite_output_dir=True,
    num_train_epochs=3, # Adjust epochs
    per_device_train_batch_size=4, # Adjust batch size based on GPU memory
    save_steps=1000,
    save_total_limit=2,
    prediction_loss_only=True,
    logging_steps=100,
)

# Use DataCollatorForLanguageModeling for Causal Language Modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False # Set to False for Causal Language Modeling (GPT-style)
)

In [8]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"], # Assumes your dataset has a 'train' split
)

trainer.train()

Step,Training Loss
100,2.220800


KeyboardInterrupt: 

In [ ]:
trainer.save_model("./final_gpt2_model")
tokenizer.save_pretrained("./final_gpt2_model")

In [ ]:
from transformers import pipeline

generator = pipeline('text-generation', model='./final_gpt2_model', tokenizer='gpt2', device=0) # use device=0 for GPU

prompt = "Instruction: Write a short, encouraging poem.\nResponse:"
result = generator(prompt, max_length=100, num_return_sequences=1)
print(result[0]['generated_text'])